<a href="https://colab.research.google.com/github/IshanChattopadhaya/C-Project/blob/main/Copy_of_Untitled6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
nlp = spacy.load("en_core_web_lg")
nlp

In [ ]:
doc = nlp("I am Jon SNow.")
doc
doc.ents

(Jon SNow,)

In [ ]:
type(doc.ents[0])

spacy.tokens.span.Span

In [ ]:
from spacy import displacy
displacy.render(doc, style ='ent', jupyter = True)

In [ ]:
import json
with open('/content/Corona2.json','r') as f:
  data = json.load(f)

In [ ]:
data['examples'][0].keys()

dict_keys(['id', 'content', 'metadata', 'annotations', 'classifications'])

In [ ]:
data['examples'][0]['content']

"While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]"

In [ ]:
data['examples'][0][ 'annotations'][0]

{'id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
 'tag_id': 'c06bd022-6ded-44a5-8d90-f17685bb85a1',
 'end': 371,
 'start': 360,
 'example_id': '18c2f619-f102-452f-ab81-d26f7e283ffe',
 'tag_name': 'Medicine',
 'value': 'Diosmectite',
 'correct': None,
 'human_annotations': [{'timestamp': '2020-03-21T00:24:32.098000Z',
   'annotator_id': 1,
   'tagged_token_id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
   'name': 'Ashpat123',
   'reason': 'exploration'}],
 'model_annotations': []}

In [ ]:
training_data = []

for example in data['examples']:
  temp_dict = {}
  temp_dict['text'] = example['content']
  temp_dict['entities'] = []
  for ann in example['annotations']:
    start = ann['start']
    end = ann['end']
    label = ann['tag_name'].upper()
    # value = ann['value']
    # temp_dict['entities'].append((start,end,value,label))
    temp_dict['entities'].append((start,end,label))
  training_data.append(temp_dict)

print(training_data[0])


{'text': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]", 'entities': [(360, 371, 'MEDICINE'), (383, 408, 'MEDICINE'), (104, 112, 'MEDICALCONDITION'), (679,

In [ ]:
training_data[0]['entities']

[(360, 371, 'MEDICINE'),
 (383, 408, 'MEDICINE'),
 (104, 112, 'MEDICALCONDITION'),
 (679, 689, 'MEDICINE'),
 (6, 23, 'MEDICINE'),
 (25, 37, 'MEDICINE'),
 (461, 470, 'MEDICALCONDITION'),
 (577, 589, 'MEDICINE'),
 (853, 865, 'MEDICALCONDITION'),
 (188, 198, 'MEDICINE'),
 (754, 762, 'MEDICALCONDITION'),
 (870, 880, 'MEDICALCONDITION'),
 (823, 833, 'MEDICINE'),
 (852, 853, 'MEDICALCONDITION'),
 (461, 469, 'MEDICALCONDITION'),
 (535, 543, 'MEDICALCONDITION'),
 (692, 704, 'MEDICINE'),
 (563, 571, 'MEDICALCONDITION')]

In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")
doc_bin = DocBin()

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
from spacy.util import filter_spans

for ex  in tqdm(training_data):
    text = ex['text']
    labels = ex['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skip")
        else:
            ents.append(span)
            #print(span)
    filter = filter_spans(ents)
    doc.ents = filter
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

100%|██████████| 31/31 [00:00<00:00, 408.52it/s]

Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip


In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

Usage: python -m spacy train [OPTIONS] CONFIG_PATH
Try 'python -m spacy train --help' for help.
╭─ Error ──────────────────────────────────────────────────────────────────────────────────────────╮
│ Invalid value for 'CONFIG_PATH': Path 'config.cfg' does not exist.                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    153.29    0.55    0.91    0.39    0.01
  7     200        499.25   3640.10   70.00   82.80   60.63    0.70
 14     400        192.16   1153.36   87.20   97.10   79.13    0.87
 22     600         72.20    471.57   93.66   97.45   90.16    0.94
 30     800        117.10    435.09   95.67   95.67   95.67    0.96
 40    1000         97.26    314.46   96.43   97.20   95.67    0.96
 51    1200         96.63    232.85   97.84   97.65   98.03    0.98
 65    1400        311.37    219.57   97.46   96.89   9

In [ ]:
nilp_ner = spacy.load("model-best")

In [ ]:
doc = nilp_ner("Hantaviruses, penicilin, usually found in rodents and shrews, were discovered in two species of bats. The Mouyassué virus (MOUV) was isolated from banana pipistrelle bats captured near Mouyassué village in Cote d'Ivoire, West Africa. The Magboi virus was isolated from hairy slit-faced bats found near the Magboi River in Sierra Leone in 2011. They are single-stranded, negative sense, RNA viruses in the Bunyaviridae family.[29][30][31][32]")

colors = {"PATHOGEN" : "#FFC0CB", "MEDICINE" : "#ADD8E6", "MEDICALCONDITION" : "#FFFF00"}
options = {"colors" : colors}

spacy.displacy.render(doc,style="ent", options = options, jupyter = True)

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gradio


In [ ]:
import spacy
import gradio as gr

In [ ]:
nlp = spacy.load("model-best")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
def predict_entities(text):
    # Process the text with your spaCy model
    doc = nlp(text)

    # Extract the predicted entities
    entities = [(ent.text, ent.label_) for ent in doc.ents]

    # Return the predicted entities
    return entities

In [ ]:
# Define an interface with a text input and an output showing the predicted entities
interface = gr.Interface(
    fn=predict_entities,
    inputs=gr.Textbox(placeholder="Enter sentence here..."),
    outputs=gr.HighlightedText(),
    #live=True,
)

# Launch the interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4c9b848771998f3974.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
!pip show gradio